<a href="https://colab.research.google.com/github/merrymira/bkktrafficdata/blob/main/1_Excel_Read_Transpose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Machine Learning in Automating Traffic Controller at the Congested Intersection in Urban Cities to enhance the Traffic Flow

##Data Pre-Processing

The original dataset consists of Excel files that do not conform to First Normal Form (1NF). As the initial format is unsuitable for training the model, this notebook outlines steps to transpose and merge the files into an appropriate format.

---------------------
**Note:** This work belongs to Marisara Satrulee, submitted on 1 June 2024 for the subject: 36123 Research Paper in Autumn 2024. For more information please contact dmarisara@gmail.com.

**Import Necessary Packages**

In [ ]:
import os
import pandas as pd
import re

In [ ]:
# prompt: read filenames from google drive folder

from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/Data_Research/Traffic_NoTranspose'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Data_Research/Traffic_NoTranspose


**List all files for tranpose and later merge into one single file**

In [ ]:
!ls

Traffic_Volumn_2023-10.xlsx  Traffic_Volumn_2023-4.xlsx  Traffic_Volumn_2023-8.xlsx
Traffic_Volumn_2023-1.xlsx   Traffic_Volumn_2023-5.xlsx  Traffic_Volumn_2023-9.xlsx
Traffic_Volumn_2023-2.xlsx   Traffic_Volumn_2023-6.xlsx
Traffic_Volumn_2023-3.xlsx   Traffic_Volumn_2023-7.xlsx


**Define a function called "extractCarType", which are the horizontal columns which will be transposed to form an 1NF table**

In [ ]:
def extractCarType(input_str):
    # Use re.match() to find the match for the first pattern
    match1 = re.match(r'^([^_]*)', input_str)

    # Use re.match() to find the match for the second pattern (after underscore but before hyphen)
    match2 = re.search(r'(?<=_)([^-_]*)', input_str)

    # Use re.match() to find the match for the third pattern (before hyphen)
    match3 = re.search(r'-(.*)', input_str)

    # Extract the substrings from the matches
    substring1 = match1.group(1) if match1 else None
    substring2 = match2.group(0) if match2 else None
    substring3 = match3.group(1) if match3 else None

    return substring1, substring2, substring3

In [ ]:
folder_path = '/content/gdrive/My Drive/Data_Research/Traffic_NoTranspose' # Replace with your actual folder path
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.xlsx')]

In [ ]:
# List all files in the folder
files

['Traffic_Volumn_2023-2.xlsx',
 'Traffic_Volumn_2023-3.xlsx',
 'Traffic_Volumn_2023-5.xlsx',
 'Traffic_Volumn_2023-7.xlsx',
 'Traffic_Volumn_2023-6.xlsx',
 'Traffic_Volumn_2023-8.xlsx',
 'Traffic_Volumn_2023-10.xlsx',
 'Traffic_Volumn_2023-9.xlsx',
 'Traffic_Volumn_2023-1.xlsx',
 'Traffic_Volumn_2023-4.xlsx']

**Difine the columns to melt (CarType)**

In [ ]:
# List of columns you want to melt
columns_to_melt = ['Car_7-9', 'Van_7-9', 'Bus_7-9',
                   'Minibus_7-9', 'Truck_7-9', '3Cycle_7-9',
                   'Car_9-17', 'Van_9-17', 'Bus_9-17',
                   'Minibus_9-17', 'Truck_9-17', '3Cycle_9-17',
                   'Car_17-19', 'Van_17-19', 'Bus_17-19',
                   'Minibus_17-19', 'Truck_17-19', '3Cycle_17-19']

**Apply the function extractCarType() to each file and rename the file by adding '_transposed' to the original name**

In [ ]:
for file in files:
    df = pd.read_excel(os.path.join(folder_path, file))

    # Reset the index to make it a column
    df = df.reset_index()

    melted_df = df.melt(id_vars='index', value_vars=columns_to_melt, var_name='Variable', value_name='Volume')
    remaining_columns = df.drop(columns=columns_to_melt)
    combined_df = pd.merge(remaining_columns, melted_df, on='index')

    combined_df['No'] = combined_df['No'].index + 1
    combined_df['CarType'] = combined_df['Variable'].apply(lambda x: extractCarType(x)[0])
    combined_df['Start_Hr'] = combined_df['Variable'].apply(lambda x: extractCarType(x)[1])
    combined_df['End_Hr'] = combined_df['Variable'].apply(lambda x: extractCarType(x)[2])

    combined_df.drop(['Variable','index'], axis=1, inplace=True)

    output_file_path = os.path.join(folder_path, file.replace('.xlsx', '') + '_transposed.xlsx')
    combined_df.to_excel(output_file_path, index=False)

In [ ]:
combined_df

,No,Date,Crossroads,Road,Total_Vol,Lat,Long,Volume,CarType,Start_Hr,End_Hr
0,1,2023-04-03,หทัยมิตร - นิมิตรใหม่,นิมิตรใหม่,32583,13.859775,100.731880,3329,Car,7,9
1,2,2023-04-03,หทัยมิตร - นิมิตรใหม่,นิมิตรใหม่,32583,13.859775,100.731880,1761,Van,7,9
2,3,2023-04-03,หทัยมิตร - นิมิตรใหม่,นิมิตรใหม่,32583,13.859775,100.731880,3,Bus,7,9
3,4,2023-04-03,หทัยมิตร - นิมิตรใหม่,นิมิตรใหม่,32583,13.859775,100.731880,2,Minibus,7,9
4,5,2023-04-03,หทัยมิตร - นิมิตรใหม่,นิมิตรใหม่,32583,13.859775,100.731880,742,Truck,7,9
...,...,...,...,...,...,...,...,...,...,...,...
661,662,2023-04-27,ณ ระนอง,รัชดาภิเษก,7945,13.717561,100.558105,321,Van,17,19
662,663,2023-04-27,ณ ระนอง,รัชดาภิเษก,7945,13.717561,100.558105,9,Bus,17,19
663,664,2023-04-27,ณ ระนอง,รัชดาภิเษก,7945,13.717561,100.558105,0,Minibus,17,19
664,665,2023-04-27,ณ ระนอง,รัชดาภิเษก,7945,13.717561,100.558105,19,Truck,17,19


**Combined all transposed Excel file into a single file called 'merged_traffic_data.xlsx'**

In [ ]:
# prompt: merge all file in files by adding one to another

folder_path = '/content/gdrive/My Drive/Data_Research/Traffic_NoTranspose'
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and '_transposed' in f]

# Initialize an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# Iterate through the files and append their data to the merged DataFrame
for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path)
    merged_df = pd.concat([merged_df, df], ignore_index=True)

# Save the merged DataFrame to a new Excel file
output_file_path = os.path.join(folder_path, 'merged_traffic_data.xlsx')
merged_df.to_excel(output_file_path, index=False)

In [ ]:
merged_df.head()

,No,Date,Crossroads,Road,Total_Vol,Lat,Long,Volume,CarType,Start_Hr,End_Hr
0,1,2023-02-01 00:00:00,ราชเทวี,พญาไท,52248.0,13.753247,100.531796,7578.0,Car,7,9
1,2,2023-02-01 00:00:00,ราชเทวี,พญาไท,52248.0,13.753247,100.531796,1372.0,Van,7,9
2,3,2023-02-01 00:00:00,ราชเทวี,พญาไท,52248.0,13.753247,100.531796,133.0,Bus,7,9
3,4,2023-02-01 00:00:00,ราชเทวี,พญาไท,52248.0,13.753247,100.531796,3.0,Minibus,7,9
4,5,2023-02-01 00:00:00,ราชเทวี,พญาไท,52248.0,13.753247,100.531796,43.0,Truck,7,9
